# Logistic Regression Customer Churn Pyspark

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [4]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,
                     header=True)

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [7]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [10]:
output = assembler.transform(data)

In [11]:
final_data = output.select('features','churn')

In [12]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [13]:
from pyspark.ml.classification import LogisticRegression

In [14]:
lr_churn = LogisticRegression(labelCol='churn')

In [15]:
fitted_churn_model = lr_churn.fit(train_churn)

In [16]:
training_sum = fitted_churn_model.summary

In [17]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                624|                624|
|   mean|0.16025641025641027|              0.125|
| stddev| 0.3671379894939084|0.33098423194731336|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [19]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [20]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.61850461202128...|[0.99022887612801...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.66476724988575...|[0.99872657033029...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.15620512990059...|[0.99971313304984...|       0.0|
|[28.0,11204.23,0....|    0|[1.99870323419810...|[0.88066085864026...|       0.0|
|[28.0,11245.38,0....|    0|[3.85150404656476...|[0.97919431922234...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.10188394052235...|[0.98372768535508...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.64156388753488...|[0.93348912763576...|       0.0|
|[29.0,13255.05,1....|    0|[4.18713805108111...|[0.98503757974240...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.67448992112662...|[0.97526499847701...|       0.0|
|[30.0,8403.78,1

In [21]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [22]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [23]:
auc

0.7767256637168142